<a href="https://colab.research.google.com/github/Equinox-Initiative/StoreQ-server/blob/master/Malai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pulling Info From Firebase

In [0]:
import math

In [0]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

cred = credentials.Certificate("/content/project-malai-firebase-adminsdk-6w258-06015654e5.json")

firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://project-malai.firebaseio.com/'
})

ValueError: ignored

In [0]:
def checkIfSameItem (userEnteredItem, actualItemName, actualItemPath):
  itemTags = db.reference(actualItemPath+"/tags").get()
  if "," in itemTags:
    tagArray = itemTags.split(",")
    for tag in tagArray:
      if userEnteredItem in tag:
        return True
  else:
    if userEnteredItem in itemTags:
      return True
  
  if userEnteredItem in actualItemName:
    return True

  return False
  

In [0]:
def getAisleWithItem (storeID, itemName):
  aislesRef = db.reference("/stores/"+storeID+"/aisles")
  aisles = aislesRef.get() #dict of aisles

  aisleIDs = list(aisles.keys())

  for aisleID in aisleIDs:
    items = db.reference("/stores/"+storeID+"/aisles/"+aisleID+"/items").get()
    actualItemNames = list(items.keys())

    for actualItem in actualItemNames:
      if checkIfSameItem(itemName, actualItem, "/stores/"+storeID+"/aisles/"+aisleID+"/items/"+actualItem):
        quantity = db.reference("/stores/"+storeID+"/aisles/"+aisleID+"/items/"+actualItem+"/quantity").get()
        return str(aisleID)# + ":" + str(quantity)
    
  return "-1"

In [0]:
def storesWithinDistance (userLatitude, userLongitude, willingTime):
  storesWithin = []

  allStores = db.reference("/stores").get()
  allStoreIDs = list(allStores.keys())

  for storeID in allStoreIDs:
    #PUT GOOGLE MAPS DISTANCE MATRIX API CALCULATIONS IN HERE - nvm
    storeLat = db.reference("/stores/"+storeID+"/lat").get()
    storeLong = db.reference("/stores/"+storeID+"/long").get()

    distance = calculate_distance(userLatitude,userLongitude,storeLat,storeLong)

    distanceInMiles = distance*69.0 #degrees to miles

    storesWithin.append(storeID)

  return storesWithin

In [0]:
def calculatePotentialTrips (userID):
  userLat = db.reference("/users/"+userID+"/latitude/").get()
  userLong = db.reference("/users/"+userID+"/longitude/").get()
  willingTimeToTravel = db.reference("/users/"+userID+"/willingTimeToTravel/")

  storesToCheck = storesWithinDistance(userLat, userLong, willingTimeToTravel)

  avaliableStores = ""

  shoppingList = db.reference("/users/"+userID+"/shoppingList/").get()
  itemNames = list(shoppingList.keys())

  for store in storesToCheck:
    itemsAvailable = []

    for itemName in itemNames:
      aisleAndQuantity = getAisleWithItem(store, itemName)

      if (aisleAndQuantity != "-1"):
         itemsAvailable.append(itemName)

    percent = float(len(itemsAvailable))/float(len(itemNames))

    storeRef = db.reference("/users/"+userID+"/potentialTrips/"+store)

    itemString = ""
    for item in itemsAvailable:
      itemString+=item+","

    storeRef.update({
        'items': itemString[:-1]
    })

    storeRef.update({
        'percentOfList': str(percent)
    })

In [0]:
calculatePotentialTrips("EcQmsQreYsXtdTj6yqrtde7xYYh2")

In [0]:
def calculate_distance(x1, y1, x2, y2):
    return math.sqrt((math.pow(x2-x1, 2)) + (math.pow(y2-y1, 2))) 

In [0]:
def getPersonAisles(personID, storeID):
    if db.reference("/users/"+personID+"/potentialTrips/"+storeID+"/items").get() is None:
      return
    else:
      items = db.reference("/users/"+personID+"/potentialTrips/"+storeID+"/items").get().split(",")
      aisles = []
      if items is None:
        return
      for item in items:
          aisles = getAisleWithItem(storeID, item)
    return aisles

In [0]:
getPersonAisles("EcQmsQreYsXtdTj6yqrtde7xYYh2", "placeholder-id")

'aisle0'

In [0]:
def getPersonAvailableTimes(personID):
    availableTimes = []
    availableTimesString = ""
    availableTimesString = db.reference("/users/"+personID+"/availableTimes").get().split(",")
    for i in availableTimesString:
      availableTimes.append(i)
    return availableTimes

In [0]:
def getPeopleScheduledInTimeSlot(storeID, timeSlot):
    users = []
    usersString = ""
    usersString = db.reference("/stores/"+storeID+"/times/"+timeSlot).get().split(",")
    for i in usersString:
        users.append(i)
    return users

In [0]:
def getNumPeopleScheduledInTimeSlot(storeID, timeSlot):
    users = []
    users = getPeopleScheduledInTimeSlot(storeID, timeSlot)
    return len(users)

In [0]:
def recommendTimes(personID, storeID, maxNumPeopleInStore, greenNumPeoplePerAisle, yellowNumPeoplePerAisle):
    timeStatusRef = db.reference("/users/"+personID+"/potentialTrips/"+storeID+"/times")
    availableTimes = []
    availableTimes = getPersonAvailableTimes(personID)
    peopleInTimeSlot = []
    userAisles = []
    userAisles.append(getPersonAisles(personID, storeID))
    print(userAisles[0])
    green = False
    for time in availableTimes:
        peopleInTimeSlot = getPeopleScheduledInTimeSlot(storeID, time)
        print(peopleInTimeSlot)
        if (getNumPeopleScheduledInTimeSlot(storeID, time) > maxNumPeopleInStore):
            timeStatusRef.update({
                str(time): "red"
            })
        else:
            print("In else statement...")
            for aisle in userAisles:
                n = 0
                for person in peopleInTimeSlot:
                    personAisles = getPersonAisles(person, storeID)
                    if aisle == personAisles:
                        n = n+1
                if n <= greenNumPeoplePerAisle:
                    gyr = True
                elif n > greenNumPeoplePerAisle and n <= yellowNumPeoplePerAisle:
                    timeStatusRef.update({
                        str(time): "yellow"
                    })
                    green = False
                    break
                else:
                    timeStatusRef.update({
                        str(time): "red"
                    })
                    green = False
                    break
            if green:
                timeStatusRef.update({
                    str(time): "green"
                })

In [0]:
calculatePotentialTrips("sjk8YXy2vFWHvjNpzi2ms0i9rZu1")
recommendTimes("sjk8YXy2vFWHvjNpzi2ms0i9rZu1", "placeholder-id", 1,1,3)

aisle1
['pvtaAiesT6MR2Vdn9afHGitJ1oR2', ' EcQmsQreYsXtdTj6yqrtde7xYYh2']
['jHpzojL6RwTfAEGWs7e23cwNciw1']
In else statement...


In [0]:
def addUserTimesToStore (userID, tripStoreID):
  timeRef = db.reference("/users/"+userID+"/trips/"+tripStoreID+"/time")
  time = timeRef.get()

  storeTimesRef = db.reference("/stores/"+tripStoreID+"/times")
  peopleInStoreAtTime = db.reference("/stores/"+tripStoreID+"/times/"+time)

  if userID not in peopleInStoreAtTime.get():
    storeTimesRef.update({
        str(time): peopleInStoreAtTime.get()+","+userID
    })

    items = list(db.reference("/users/"+userID+"/trips/"+tripStoreID+"/items").get().keys)

    alreadyAddedAisles = []
    for item in items:
      aisle = getAisleWithItem(tripStoreID, item)
      if aisle not in alreadyAddedAisles:
        alreadyAddedAisles.append(aisle)

        aisleQuantityRef = db.reference("/store/"+tripStoreID+"/aisles/"+aisle+"/times/"+time)
        quantity = aisleQuantityRef.get()
        aisleQuantityRef.update({
          str(time): quantity+1
        })

  

In [0]:
addUserTimesToStore("pvtaAiesT6MR2Vdn9afHGitJ1oR2", "placeholder-id")